
## Resumen ejecutivo

Este proyecto trata de **predecir las ventas diarias (unit\_sales) de cada producto en cada tienda** de la cadena minorista “Corporación Favorita” para **optimizar inventarios**, **evitar desabastecimientos y sobrestock**, y **mejorar la planificación de promociones**. La variable objetivo es **unit\_sales** y la métrica de evaluación es la **Normalized Weighted Root Mean Squared Logarithmic Error (NWRMSLE)**, la cual penaliza de forma logarítmica los errores y pondera más los productos perecibles ([Kaggle][1]). Los datos provienen de un histórico de ventas diarias (2012–2017) extraídos del sistema POS de Favorita y puestos a disposición en Kaggle, junto con información complementaria de transacciones, precios del petróleo, feriados y metadatos de tiendas y productos ([GitHub][2], [Kaggle][3]).

---

## 1. ¿Cuál es el problema de negocio o la pregunta de investigación?

**Problema de negocio:** Prever la demanda diaria de cada combinación (tienda, producto) para minimizar costos de **stock‑out** (pérdida de ventas) y **overstock** (costos de almacenamiento y caducidad) en la cadena minorista ([GitHub][4]).
**Pregunta de investigación:** ¿Cómo varían las ventas unitarias por tienda y producto día a día, y cómo pueden optimizarse los niveles de inventario y las promociones basándose en esas predicciones? ([GitHub][2])
**Métrica a predecir / explicar:**

* **unit\_sales**: número de unidades vendidas (puede ser entero o decimal, p. ej. kg de queso).
* **Métrica de evaluación:** NWRMSLE (Normalized Weighted Root Mean Squared Logarithmic Error), que penaliza logarítmicamente las discrepancias y aplica un peso mayor a productos perecibles (1.25 vs. 1.0) ([Kaggle][1]).

---

## 2. ¿De dónde vienen estos datos?

1. **Origen**:

   * Datos proporcionados por **Corporación Favorita**, cadena de supermercados de Ecuador, y publicados en formato de competencia en **Kaggle** ([Kaggle][3], [Kaggle][1]).
2. **Frecuencia de actualización**:

   * **Histórico de ventas diarias** desde 2012 hasta 2017; el conjunto de **entrenamiento** abarca hasta el 31 de agosto de 2017, y el de **prueba** corresponde al periodo 16 al 31 de agosto de 2017 ([GitHub][2]).
   * Datos adicionales (transacciones, precios de crudo, feriados) también se registran **diariamente**.
3. **Responsable de la generación**:

   * La **división de análisis de datos** de Corporación Favorita recoge la información desde su sistema POS.
   * Kaggle actúa como repositorio y moderador de la competencia, pero no actualiza el dataset: es un **snapshot estático**.

---

**Fixing tables and adding notes**

The user has requested that I correct tables and add markdown citations for notes regarding missing information and explanations for each table. They’ve requested corrections for sections 3.1 through 3.7, starting with train.csv and holidays\_events.csv. It looks like I’ll fix typos, add relevant citations, and complete missing notes. I’ll also need to include a new table for sample\_submission.csv since it’s currently missing. The corrected markdown will include an updated subsection for the new table as well.

**Updating tables with citations**

The user wants me to update each table by filling missing information like units, domains, and a "Notas" column, and adding citations in markdown footnotes. To make it clearer, I’ll use markdown footnotes like \[^1] for citations at the end of each table, referencing sources like Kaggle or Medium. For example, I'll add "Nota" rows with their sources in tables, and include "3.8 sample\_submission.csv" as a new subplot. The final markdown should have all corrections and citations properly placed.


Thought for 6 seconds


## 3. ¿Qué significan cada una de las columnas?

A continuación el **diccionario de datos** de los principales archivos disponibles:

### 3.1 `train.csv`

| Columna       | Tipo     | Unidad                | Descripción                                                                                                | Valores permitidos / Dominio esperado |
| ------------- | -------- | --------------------- | ---------------------------------------------------------------------------------------------------------- | ------------------------------------- |
| `id`          | entero   | —                     | Identificador único de la fila.                                                                            | Solo etiqueta                         |
| `date`        | fecha    | YYYY‑MM‑DD            | Fecha de la venta.                                                                                         | Fechas entre 2013‑01‑01 y 2017‑08‑15  |
| `store_nbr`   | entero   | —                     | Código de tienda.                                                                                          | 1–54                                  |
| `item_nbr`    | entero   | —                     | Código de producto.                                                                                        | 1–\~4400                              |
| `unit_sales`  | float    | unidades (o kg, etc.) | Ventas diarias. Valores ≥ 0 para ventas; valores < 0 indican devoluciones.                                 | Numérico, decimales permitidos        |
| `onpromotion` | booleano | —                     | Indica si el producto estaba en promoción ese día. NaN (≈ 16 % de los registros) se interpreta como False. | True / False                          |

> **Nota**:
>
> * Los datos **no** incluyen filas donde `unit_sales = 0` (no se registró venta ni devolución), por lo que faltan combinaciones tienda–artículo en días sin movimiento.
> * Un pequeño número de `item_nbr` aparece en entrenamiento pero **no** en prueba.
> * Aproximadamente el 16 % de los `onpromotion` son NaN y deben tratarse como False.
>   ([Kaggle Data][3], [Medium][5])

---

### 3.2 `test.csv`

| Columna       | Tipo     | Unidad     | Descripción                                                        |
| ------------- | -------- | ---------- | ------------------------------------------------------------------ |
| `id`          | entero   | —          | Identificador único de la fila de prueba.                          |
| `date`        | fecha    | YYYY‑MM‑DD | Fechas a predecir: del 2017‑08‑16 al 2017‑08‑31.                   |
| `store_nbr`   | entero   | —          | Código de tienda.                                                  |
| `item_nbr`    | entero   | —          | Código de producto.                                                |
| `onpromotion` | booleano | —          | Indica si el producto estará en promoción en el periodo de prueba. |

> **Nota**:
>
> * Incluye algunos `item_nbr` que **no** aparecen en `train.csv`.
> * La división pública/privada de la leaderboard se basa en tiempo; todos los ítems de la pública están en la privada.
>   ([Kaggle Data][3])

---

### 3.3 `stores.csv`

| Columna     | Tipo   | Descripción                                           |
| ----------- | ------ | ----------------------------------------------------- |
| `store_nbr` | entero | Código de tienda.                                     |
| `city`      | texto  | Ciudad donde está ubicada la tienda.                  |
| `state`     | texto  | Provincia o estado.                                   |
| `type`      | texto  | Tipo de tienda (A, B, C, … según tamaño/formato).     |
| `cluster`   | entero | Grupo asignado tras análisis de clústeres de tiendas. |

> **Nota**:
>
> * `cluster` agrupa tiendas con comportamiento de ventas similar.
>   ([Kaggle Data][3])

---

### 3.4 `items.csv`

| Columna      | Tipo   | Descripción                                                                                |
| ------------ | ------ | ------------------------------------------------------------------------------------------ |
| `item_nbr`   | entero | Código de producto.                                                                        |
| `family`     | texto  | Categoría general (por ejemplo, “BEVERAGES”, “DAIRY”).                                     |
| `class`      | entero | Subcategoría o clase específica dentro de la familia.                                      |
| `perishable` | entero | 1 si el producto es perecible (peso 1.25 en la métrica de evaluación), 0 si no (peso 1.0). |

> **Nota**:
>
> * Productos con `perishable=1` tienen mayor peso en la métrica (1.25 vs. 1.0).
>   ([Kaggle Data][3])

---

### 3.5 `transactions.csv`

| Columna        | Tipo   | Descripción                                         |
| -------------- | ------ | --------------------------------------------------- |
| `date`         | fecha  | Fecha de la transacción.                            |
| `store_nbr`    | entero | Código de tienda.                                   |
| `transactions` | entero | Número total de transacciones diarias en la tienda. |

> **Nota**:
>
> * Solo cubre el **periodo de entrenamiento**.
>   ([Kaggle Data][3])

---

### 3.6 `oil.csv`

| Columna      | Tipo  | Descripción                                                                          |
| ------------ | ----- | ------------------------------------------------------------------------------------ |
| `date`       | fecha | Fecha de referencia.                                                                 |
| `dcoilwtico` | float | Precio diario del crudo West Texas Intermediate (WTI), influye en costos logísticos. |

> **Nota**:
>
> * Contiene precios tanto para **entrenamiento** como **prueba**; Ecuador es muy sensible a choques en el crudo.
>   ([Kaggle Data][3])

---

### 3.7 `holidays_events.csv`

| Columna       | Tipo  | Descripción                                                                    |
| ------------- | ----- | ------------------------------------------------------------------------------ |
| `date`        | fecha | Fecha del feriado o evento.                                                    |
| `type`        | texto | “Holiday”, “Event”, “Transfer”, “Bridge”, “Work Day”, etc.                     |
| `locale`      | texto | Ámbito del evento (“National”, “Local”, “Regional”).                           |
| `locale_name` | texto | Nombre de la ciudad o región si `locale` = “Local” o “Regional”.               |
| `description` | texto | Nombre descriptivo del feriado o evento.                                       |
| `transferred` | bool  | True si la fecha oficial se trasladó (por ejemplo, un feriado movido a lunes). |

> **Nota**:
>
> * Para saber cuándo se celebró realmente un feriado trasladado, busque la fila con `transferred = True`.
>   ([Kaggle Data][3])

---

### 3.8 `sample_submission.csv`

| Columna      | Tipo   | Descripción                                              |
| ------------ | ------ | -------------------------------------------------------- |
| `id`         | entero | Debe coincidir con los `id` de `test.csv`.               |
| `unit_sales` | float  | Predicción de `unit_sales` para cada fila de `test.csv`. |

> **Nota**:
>
> * Debe comprimirse (ZIP) antes de subir al portal de Kaggle para no exceder límite de tamaño.
>   ([Kaggle Data][3])

---

[1]: https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting/overview/evaluation
[3]: https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data
[5]: https://kishan-122.medium.com/kaggle-corporaci%C3%B3n-favorita-grocery-sales-forecasting-b36e0caa0a9f
[6]: https://medium.com/%40baakwadegraftmiah/time-series-forecasting-predictive-model-building-for-store-sales-at-favorita-grocery-retailer-b5ef9d262b7b


> **Nota 📝**  
> Estoy utilizando **tablas con Rich** para representar datos debido a su estética clara y eficiente en terminales.  
> No uso Pandas por limitaciones de recursos, ya que cargar archivos grandes en memoria no es viable en mi entorno actual.  
> Además, esto forma parte de un entrenamiento personal para optimizar recursos y desarrollar habilidades para trabajar en **entornos ajustados**.


In [9]:
import dask.dataframe as dd
from rich.table import Table
from rich.console import Console

# Rutas a los archivos CSV
paths = {
    "train": "../data/raw/train.csv",
    "test": "../data/raw/test.csv",
    "sample_submission": "../data/raw/sample_submission.csv",
    "stores": "../data/raw/stores.csv",
    "items": "../data/raw/items.csv",
    "oil": "../data/raw/oil.csv",
    "holidays_events": "../data/raw/holidays_events.csv",
    "transactions": "../data/raw/transactions.csv",
}

console = Console()

# 1) Tabla de dimensiones
dim_table = Table(title="Dimensiones de DataFrames")
dim_table.add_column("DataFrame", style="bold cyan")
dim_table.add_column("Filas", justify="right", style="magenta")
dim_table.add_column("Columnas", justify="right", style="green")

for name, path in paths.items():
    try:
        df = dd.read_csv(path)
        rows = df.shape[0].compute()
        cols = df.shape[1]
        filas_fmt = f"{rows:,}"
        dim_table.add_row(name, filas_fmt, str(cols))
    except Exception as e:
        dim_table.add_row(name, "Error", str(e))

console.print(dim_table)


# 2) Tabla de tipos de cada columna
dtype_table = Table(title="Tipos de datos de columnas por DataFrame")
dtype_table.add_column("DataFrame", style="bold cyan")
dtype_table.add_column("Columna", style="yellow")
dtype_table.add_column("Tipo de dato", style="green")

for name, path in paths.items():
    try:
        df = dd.read_csv(path)
        # df.dtypes es un Series con index=columnas y values=dtypes
        for col, dtype in df.dtypes.items():
            dtype_table.add_row(name, col, str(dtype))
    except Exception as e:
        dtype_table.add_row(name, "-", f"Error: {e}")

console.print(dtype_table)


c:\Users\spinz\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


          Dimensiones de DataFrames           
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ DataFrame         ┃       Filas ┃ Columnas ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ train             │ 125,497,040 │        6 │
│ test              │   3,370,464 │        5 │
│ sample_submission │   3,370,464 │        2 │
│ stores            │          54 │        5 │
│ items             │       4,100 │        4 │
│ oil               │       1,218 │        2 │
│ holidays_events   │         350 │        6 │
│ transactions      │      83,488 │        3 │
└───────────────────┴─────────────┴──────────┘

     Tipos de datos de columnas por DataFrame      
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ DataFrame         ┃ Columna      ┃ Tipo de dato ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ train             │ id           │ int64        │
│ train             │ date         │ string       │
│ train             │ store_nbr    │ int64        │
│ train             │ item_nbr     │ int64        │
│ train             │ unit_sales   │ float64      │
│ train             │ onpromotion  │ float64      │
│ test              │ id           │ int64        │
│ test              │ date         │ string       │
│ test              │ store_nbr    │ int64        │
│ test              │ item_nbr     │ int64        │
│ test              │ onpromotion  │ bool         │
│ sample_submission │ id           │ int64        │
│ sample_submission │ unit_sales   │ int64        │
│ stores            │ store_nbr    │ int64        │
│ stores            │ city         │ string       │
│ stores            │ state        │ string       │
│ stores            │ type         │ string       │
│ stores            │ cluster      │ int64        │
│ items             │ item_nbr     │ int64        │
│ items             │ family       │ string       │
│ items             │ class        │ int64        │
│ items             │ perishable   │ int64        │
│ oil               │ date         │ string       │
│ oil               │ dcoilwtico   │ float64      │
│ holidays_events   │ date         │ string       │
│ holidays_events   │ type         │ string       │
│ holidays_events   │ locale       │ string       │
│ holidays_events   │ locale_name  │ string       │
│ holidays_events   │ description  │ string       │
│ holidays_events   │ transferred  │ bool         │
│ transactions      │ date         │ string       │
│ transactions      │ store_nbr    │ int64        │
│ transactions      │ transactions │ int64        │
└───────────────────┴──────────────┴──────────────┘

## calidad de los datos

In [1]:
import dask.dataframe as dd
from rich.table import Table
from rich.console import Console
import numpy as np

# Rutas a los archivos CSV
paths = {
    "train": "../data/raw/train.csv",
    "test": "../data/raw/test.csv",
    "sample_submission": "../data/raw/sample_submission.csv",
    "stores": "../data/raw/stores.csv",
    "items": "../data/raw/items.csv",
    "oil": "../data/raw/oil.csv",
    "holidays_events": "../data/raw/holidays_events.csv",
    "transactions": "../data/raw/transactions.csv",
}

console = Console()

# Tabla para resultados de revisión
result_table = Table(title="Resumen de calidad por columna")
result_table.add_column("DataFrame", style="bold cyan")
result_table.add_column("Columna", style="yellow")
result_table.add_column("NaN", justify="right", style="magenta")
result_table.add_column("-inf", justify="right", style="red")
result_table.add_column("Duplicados", justify="right", style="blue")
result_table.add_column("Negativos", justify="right", style="green")
result_table.add_column("Únicos", justify="right", style="white")  # Nueva columna para valores distintos

for name, path in paths.items():
    try:
        df = dd.read_csv(path, assume_missing=True)
        # Número total de filas
        total_rows = df.shape[0].compute()

        for col in df.columns:
            s = df[col]

            # Conteo NaN
            n_nan = s.isna().sum().compute()

            # Conteo -inf
            try:
                n_ninf = (s == -np.inf).sum().compute()
            except Exception:
                n_ninf = 0

            # Conteo duplicados
            try:
                dup_series = df[[col]].duplicated(subset=[col])
                n_dup = dup_series.sum().compute()
            except Exception:
                try:
                    uniq = s.nunique().compute()
                    n_dup = total_rows - uniq
                except Exception:
                    n_dup = 0

            # Conteo negativos
            try:
                n_neg = (s < 0).sum().compute()
            except Exception:
                n_neg = 0

            # Conteo valores únicos
            try:
                n_unique = s.nunique().compute()
            except Exception:
                n_unique = 0

            result_table.add_row(
                name,
                col,
                f"{n_nan:,}",
                f"{n_ninf:,}",
                f"{n_dup:,}",
                f"{n_neg:,}",
                f"{n_unique:,}",
            )

    except Exception as e:
        result_table.add_row(name, "-", "-", "-", "-", "-", str(e))

console.print(result_table)


c:\Users\spinz\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\spinz\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\spinz\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\spinz\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\spinz\anaconda3\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, 

                                 Resumen de calidad por columna                                 
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ DataFrame         ┃ Columna      ┃        NaN ┃ -inf ┃  Duplicados ┃ Negativos ┃      Únicos ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ train             │ id           │          0 │    0 │           0 │         0 │ 125,497,040 │
│ train             │ date         │          0 │    0 │ 125,495,356 │         0 │       1,684 │
│ train             │ store_nbr    │          0 │    0 │ 125,496,986 │         0 │          54 │
│ train             │ item_nbr     │          0 │    0 │ 125,493,004 │         0 │       4,036 │
│ train             │ unit_sales   │          0 │    0 │ 125,238,566 │     7,795 │     258,474 │
│ train             │ onpromotion  │ 21,657,651 │    0 │ 125,497,038 │         0 │           2 │
│ test              │ id           │          0 │    0 │           0 │         0 │   3,370,464 │
│ test              │ date         │          0 │    0 │   3,370,448 │         0 │          16 │
│ test              │ store_nbr    │          0 │    0 │   3,370,410 │         0 │          54 │
│ test              │ item_nbr     │          0 │    0 │   3,366,563 │         0 │       3,901 │
│ test              │ onpromotion  │          0 │    0 │   3,370,462 │         0 │           2 │
│ sample_submission │ id           │          0 │    0 │           0 │         0 │   3,370,464 │
│ sample_submission │ unit_sales   │          0 │    0 │   3,370,463 │         0 │           1 │
│ stores            │ store_nbr    │          0 │    0 │           0 │         0 │          54 │
│ stores            │ city         │          0 │    0 │          32 │         0 │          22 │
│ stores            │ state        │          0 │    0 │          38 │         0 │          16 │
│ stores            │ type         │          0 │    0 │          49 │         0 │           5 │
│ stores            │ cluster      │          0 │    0 │          37 │         0 │          17 │
│ items             │ item_nbr     │          0 │    0 │           0 │         0 │       4,100 │
│ items             │ family       │          0 │    0 │       4,067 │         0 │          33 │
│ items             │ class        │          0 │    0 │       3,763 │         0 │         337 │
│ items             │ perishable   │          0 │    0 │       4,098 │         0 │           2 │
│ oil               │ date         │          0 │    0 │           0 │         0 │       1,218 │
│ oil               │ dcoilwtico   │         43 │    0 │         220 │         0 │         998 │
│ holidays_events   │ date         │          0 │    0 │          38 │         0 │         312 │
│ holidays_events   │ type         │          0 │    0 │         344 │         0 │           6 │
│ holidays_events   │ locale       │          0 │    0 │         347 │         0 │           3 │
│ holidays_events   │ locale_name  │          0 │    0 │         326 │         0 │          24 │
│ holidays_events   │ description  │          0 │    0 │         247 │         0 │         103 │
│ holidays_events   │ transferred  │          0 │    0 │         348 │         0 │           2 │
│ transactions      │ date         │          0 │    0 │      81,806 │         0 │       1,682 │
│ transactions      │ store_nbr    │          0 │    0 │      83,434 │         0 │          54 │
│ transactions      │ transactions │          0 │    0 │      78,495 │         0 │       4,993 │
└───────────────────┴──────────────┴────────────┴──────┴─────────────┴───────────┴─────────────┘

## 1. Observaciones y Recomendaciones.

### 1.1 `train` (columna `onpromotion`)
- Se detectan **21 657 651** valores `NaN` en `onpromotion` (≈ 17 % del total).
- Según la especificación original, estos `NaN` deben interpretarse como `False`.  
  **Acción recomendada:** reemplazar todos los `NaN` por `False` para conservar la granularidad temporal y evitar la pérdida de registros.

### 1.2 `oil` (columna `dcoilwtico`)
- Existen **43** valores `NaN` y **220** duplicados en la serie de precios del crudo.
- Dadas las implicaciones logísticas y macroeconómicas, es preferible **volver a descargar** la fuente histórica de WTI desde una API oficial (por ejemplo, EIA o Yahoo Finance) y rellenar de forma confiable, antes que eliminar o imputar con medias que podrían sesgar la serie.

> **Observación senior:** siempre que sea posible, mantener la continuidad de las series temporales originales y documentar la fuente y método de imputación.  

---

## 2. Exploración de duplicados y cobertura temporal

A partir de los duplicados en las columnas de fecha (`date`), tienda (`store_nbr`) e ítem (`item_nbr`), podemos inferir:

1. **Cobertura temporal y geográfica**  
   - El conjunto `train` abarca **1 684 días** de datos (≈ 4,6 años).  
   - Los registros provienen de **54 tiendas** distintas.  
2. **Variedad de productos**  
   - Se comercializan **4 036 ítems** diferentes.  
3. **Volumen de ventas**  
   - Se registraron **258 474** unidades vendidas.  
   - De ellas, **7 795** corresponden a devoluciones (`unit_sales` < 0).  

**Acción recomendada:**  
- **Eliminar devoluciones** (`unit_sales` < 0) si el modelo se centra en predicción de demanda neta.  
- **Registrar** aparte el índice de devoluciones para futuros análisis de comportamiento post‑venta.

---

## 3. Observaciones por dataset

### 3.1 `test`
- Solo cubre **16 días** (2017‑08‑16 a 2017‑08‑31).  
- Incluye **3 901** ítems en **54** tiendas.  
- **Brecha de validación:** falta un conjunto de validación independiente.  
  **Recomendación:** reservar el 15 % de la ventana `train` como “validación” y, a largo plazo, ampliar el periodo de prueba.

### 3.2 `stores`
- Existen **54 tiendas**, distribuidas en **22 ciudades** y **16 estados**.  
- Se identifican **5 tipos** de tienda (`type`) y **17 clusters** de comportamiento.  
  **Valor agregado:** los clusters pueden usarse como variable categórica para capturar patrones de venta similares.

### 3.3 `items`
- Catálogo de **4 100** ítems.  
- La columna `family` (33 categorías) y `class` (337 subclases) permite análisis jerárquico de productos.  
- Los perecibles (`perishable = 1`) tienen mayor peso en la métrica de evaluación y merecen tratamiento especial (por caducidad y promociones).

### 3.4 `oil`
- Serie de precios WTI con **1 218** fechas únicas y **43** valores faltantes.  
- **Recomendación:** reponer datos desde una fuente autorizada y revisar si existe correlación significativa con las ventas regionales (ej., coste de transporte).

### 3.5 `holidays_events`
- **312** días de eventos y feriados (tipos: Holiday, Event, Transfer, Bridge, Work Day…).  
- Los festivos trasladados (`transferred = True`) deben vincularse con su fecha oficial real.  
  **Oportunidad:** explorar efectos “pre‑holiday” y “post‑holiday” en la demanda — algunos productos pueden anticiparse o atrasarse en las ventas.

### 3.6 `transactions`
- Registra **4 993** valores de transacciones diarias por tienda.  
- **Insight clave:** la variable `transactions` es un proxy de afluencia de clientes y puede mejorar la predicción de demanda cuando se combina con promociones y feriados.

---

## 4. Conclusiones y siguientes pasos

- **Imputación y alineación temporal:** completar `onpromotion` y `dcoilwtico` antes de procesar series.  
- **Segmentación y clustering:** aprovechar los clusters de tienda y las jerarquías de producto (`family`/`class`) para modelos de efecto fijo o embeddings.  
- **Construcción de validación robusta:** separar un periodo de validación temporal (15 %) y considerar validación cruzada temporal “sliding window”.  
- **Feature engineering:**  
  - Indicadores de feriado (pre, durante, post).  
  - Lags y medias móviles de `unit_sales` y `transactions`.  
  - Variables macro (precio del petróleo, indicadores económicos).  



Se realizaron las observaciones principales junto con sus respectivas recomendaciones. Además, me gustaría plantear una serie de preguntas adicionales que podrían ser de interés para los directivos, ya que pueden aportar valor estratégico a la toma de decisiones.

---

## Preguntas Clave para Profundizar en el Análisis

1. **¿Qué productos no se están vendiendo?**
   Identificar productos con baja rotación o sin ventas puede ayudar a optimizar el inventario y reducir costos.

2. **¿Qué productos se venden más durante los días festivos?**
   Esta información permite preparar campañas promocionales enfocadas y mejorar la logística en fechas clave.

3. **¿Los días festivos representan un aumento o disminución en las compras?**
   Analizar el comportamiento del consumidor en días festivos permite planificar mejor el personal y los recursos.

4. **¿En qué ciudades se vende más y en cuáles menos?**
   Esto puede revelar oportunidades de expansión o necesidad de reestructuración en ciertas regiones.

5. **¿Qué variables están fuertemente correlacionadas?**
   Explorar correlaciones puede descubrir factores clave que influyen en las ventas, como precios, promociones, ubicación, etc.

6. **¿Qué días de la semana se venden más productos?**
   Esta información permite optimizar los horarios de operación, distribución y atención al cliente.

7. **¿Aumenta la cantidad de productos vendidos cuando están en promoción?**
   Evaluar el impacto real de las promociones ayuda a diseñar estrategias comerciales más efectivas.

8. **¿Cuáles son los productos que pasan más tiempo en promoción?**
   Esto puede indicar problemas de rotación o sobreoferta, y permite evaluar la eficiencia de las promociones.

9. **¿Cuál es el estado con más sucursales y cuál tiene menos?**
   Entender la distribución geográfica del negocio facilita la toma de decisiones sobre expansión o consolidación.
